# Objective

1. Process the  trips data (zone-based,link-based,point-based, travel-time)


2. Collect unifished trips data


3. Set a maximum waiting time


4. Set a repositioning location

In [74]:
import pandas as pd

from shapely.geometry import Point, Polygon

import geopandas as gp

import numpy as np

import random

import pulp

import folium

import networkx as nx


In [75]:
def Check_zones(pnt,polys):
    
    key='None'
    
    for k, geom in polys.items():
        
        if pnt.within(geom):
            
            key=k
            
            break
            
    return key

def Check_links(pnt,zone,Zone_Link,Link_middle):
    
    Dic={}
    
    for link in Zone_Link[zone]:
        
        middle_pnt=Link_middle[link]
        
        Dic[link]=pnt.distance(middle_pnt)
        
    return min(Dic, key=Dic.get)

    
def Get_path(G,Points_link,Point_coordinate,source,target):
    
    link_path=list()
    
    try:
    
        path=nx.shortest_path(G, source=source, target=target,weight='weight')

        shortest_dis=nx.shortest_path_length(G, source=source, target=target,weight='weight')

        for i in range(1,len(path),1):

            pnts=path[i-1]+"&"+path[i]

            link_path.append(Points_link[pnts])
        
    except:
        
        shortest_dis=(Point_coordinate[source].distance(Point_coordinate[target])*111000)*1.3
    
    return link_path,shortest_dis

def Get_travel_time(dis,speed):
    
    return int(dis/speed)

In [76]:
'''Param'''

s_sec=25200

e_sec=36000

# speed = 10 m/seconds

speed=10

In [77]:
'''Load data'''

'''Zone-related data'''

Taxi_Zones=np.load('./Data/NYC_Zones/Taxi_Zones.npy',allow_pickle=True).item()

Zone_list=np.load('./Data/NYC_Zones/Zone_list.npy',allow_pickle=True)

Zone_Center=np.load('./Data/NYC_Zones/Zone_Center.npy',allow_pickle=True).item()

Zone_Link=np.load('./Data/NYC_Zones/Zone_Link.npy',allow_pickle=True).item()

'''Link-related data'''

Link_middle=np.load('./Data/NYC_Zones/Link_middle.npy',allow_pickle=True).item()

Link_Point=np.load('./Data/NYC_Zones/Link_Point.npy',allow_pickle=True).item()

'''Point-related data'''

Points_list=np.load('./Data/NYC_Zones/Points_list.npy',allow_pickle=True)

Points_link=np.load('./Data/NYC_Zones/Points_link.npy',allow_pickle=True).item()

Point_coordinate=np.load('./Data/NYC_Zones/Point_coordinate.npy',allow_pickle=True).item()

'''GeoSeries Object'''

polys = gp.GeoSeries(Taxi_Zones)

'''Road network Object'''

G = nx.Graph()

G.add_nodes_from(Points_list)

G.add_weighted_edges_from(list(Link_Point.values()))


In [78]:
'''1. Processing Order data'''

'''Link based and point based'''

Order_df=pd.read_csv('./Data/NewYork_New/pick_day_1.csv',header=None,names=['Pick_Second','Trip_Duration',\
                                                     'Pickup_Latitude','Pickup_Longitude',\
                                                     'Dropoff_Latitude','Dropoff_Longitude',\
                                                     'Trip_Distance','Euclidean','Manhattan'])

Order_df['Pickup_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),polys),axis=1)

Order_df['Dropoff_Zone']=Order_df.apply(lambda x:Check_zones(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),polys),axis=1)

Order_df=Order_df.loc[(Order_df['Pickup_Zone']!='None')&(Order_df['Dropoff_Zone']!='None')&(Order_df['Pick_Second']>=s_sec)&(Order_df['Pick_Second']<e_sec)]

Order_df=Order_df.sort_values(by=['Pick_Second'])

Order_df=Order_df.rename(columns={'Pick_Second':'Arrive_Second'})

Order_df=Order_df.reset_index(drop=True)

Order_df['Order_id']=['O'+str(i) for i in Order_df.index]

Order_df['Driver_id']='Unserved'

Order_df=Order_df[['Order_id','Driver_id','Arrive_Second',\
                   'Pickup_Latitude','Pickup_Longitude',\
                   'Dropoff_Latitude','Dropoff_Longitude',\
                    'Pickup_Zone','Dropoff_Zone']]

Order_df['Pickup_Link']=Order_df.apply(lambda x:Check_links(Point(x['Pickup_Longitude'],x['Pickup_Latitude']),x['Pickup_Zone'],Zone_Link,Link_middle),axis=1)

Order_df['Dropoff_link']=Order_df.apply(lambda x:Check_links(Point(x['Dropoff_Longitude'],x['Dropoff_Latitude']),x['Dropoff_Zone'],Zone_Link,Link_middle),axis=1)

Order_df['Pickup_Point']=Order_df.apply(lambda x:Link_Point[x['Pickup_Link']][0],axis=1)

Order_df['Dropoff_Point']=Order_df.apply(lambda x:Link_Point[x['Dropoff_link']][1],axis=1)

Order_df['Travel_dis']=Order_df.apply(lambda x:Get_path(G,Points_link,Point_coordinate,x['Pickup_Point'],x['Dropoff_Point'])[1],axis=1)

Order_df['Travel_time']=Order_df.apply(lambda x:Get_travel_time(x['Travel_dis'],speed),axis=1)

# Order_df.to_csv('./Data/NYC_Trips/Order_df.csv')

Order_df



,Order_id,Driver_id,Arrive_Second,Pickup_Latitude,Pickup_Longitude,Dropoff_Latitude,Dropoff_Longitude,Pickup_Zone,Dropoff_Zone,Pickup_Link,Dropoff_link,Pickup_Point,Dropoff_Point,Travel_dis,Travel_time
0,O0,Unserved,25200,40.748592,-73.987480,40.764927,-73.983566,Zone_43,Zone_42,Link_1315,Link_2258,Point_4026,Point_3620,7895.687103,789
1,O1,Unserved,25202,40.767056,-73.953827,40.755241,-73.975853,Zone_30,Zone_40,Link_654,Link_7218,Point_1229,Point_6444,9254.027214,925
2,O2,Unserved,25203,40.760441,-73.991577,40.738243,-73.998856,Zone_8,Zone_16,Link_770,Link_10174,Point_392,Point_2397,9306.546796,930
3,O3,Unserved,25206,40.756618,-73.989731,40.769596,-73.954613,Zone_18,Zone_30,Link_2452,Link_6935,Point_3656,Point_2089,13856.682323,1385
4,O4,Unserved,25206,40.725529,-74.005234,40.816872,-73.941338,Zone_17,Zone_6,Link_6016,Link_836,Point_2442,Point_4502,39020.943755,3902
5,O5,Unserved,25209,40.788265,-73.976349,40.779099,-73.944725,Zone_61,Zone_68,Link_9446,Link_398,Point_5876,Point_2183,10405.956684,1040
6,O6,Unserved,25209,40.766899,-73.983192,40.767876,-73.993156,Zone_8,Zone_9,Link_1307,Link_3590,Point_197,Point_3639,3479.745845,347
7,O7,Unserved,25209,40.743622,-73.984024,40.748802,-73.991043,Zone_43,Zone_46,Link_5730,Link_3338,Point_138,Point_1879,3456.041402,345
8,O8,Unserved,25213,40.774574,-73.957504,40.778034,-73.974724,Zone_59,Zone_55,Link_6594,Link_9977,Point_4934,Point_4013,7173.874987,717
9,O9,Unserved,25216,40.746571,-74.001511,40.764679,-73.970932,Zone_10,Zone_60,Link_4938,Link_260,Point_5199,Point_4381,14413.362063,1441


In [80]:
Order_df=pd.read_csv('./Data/NYC_Trips/Order_df.csv')

Order_df=Order_df.drop(columns=['Unnamed: 0'])



Order_df

,Order_id,Driver_id,Arrive_Second,Pickup_Latitude,Pickup_Longitude,Dropoff_Latitude,Dropoff_Longitude,Pickup_Zone,Dropoff_Zone,Pickup_Link,Dropoff_link,Pickup_Point,Dropoff_Point,Travel_dis,Travel_time
0,O0,Unserved,25200,40.748592,-73.987480,40.764927,-73.983566,Zone_43,Zone_42,Link_1315,Link_2258,Point_4026,Point_3620,7895.687103,789
1,O1,Unserved,25202,40.767056,-73.953827,40.755241,-73.975853,Zone_30,Zone_40,Link_654,Link_7218,Point_1229,Point_6444,9254.027214,925
2,O2,Unserved,25203,40.760441,-73.991577,40.738243,-73.998856,Zone_8,Zone_16,Link_770,Link_10174,Point_392,Point_2397,9306.546796,930
3,O3,Unserved,25206,40.756618,-73.989731,40.769596,-73.954613,Zone_18,Zone_30,Link_2452,Link_6935,Point_3656,Point_2089,13856.682323,1385
4,O4,Unserved,25206,40.725529,-74.005234,40.816872,-73.941338,Zone_17,Zone_6,Link_6016,Link_836,Point_2442,Point_4502,39020.943755,3902
5,O5,Unserved,25209,40.788265,-73.976349,40.779099,-73.944725,Zone_61,Zone_68,Link_9446,Link_398,Point_5876,Point_2183,10405.956684,1040
6,O6,Unserved,25209,40.766899,-73.983192,40.767876,-73.993156,Zone_8,Zone_9,Link_1307,Link_3590,Point_197,Point_3639,3479.745845,347
7,O7,Unserved,25209,40.743622,-73.984024,40.748802,-73.991043,Zone_43,Zone_46,Link_5730,Link_3338,Point_138,Point_1879,3456.041402,345
8,O8,Unserved,25213,40.774574,-73.957504,40.778034,-73.974724,Zone_59,Zone_55,Link_6594,Link_9977,Point_4934,Point_4013,7173.874987,717
9,O9,Unserved,25216,40.746571,-74.001511,40.764679,-73.970932,Zone_10,Zone_60,Link_4938,Link_260,Point_5199,Point_4381,14413.362063,1441
